# MICROTONAL EXPRESSION AND MIDI

Introduction to composing with harmonics, microtones, just intervals.

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import math
from fractions import Fraction
from musx import Score, Note, MidiEvent, Seq, MidiFile, Cycle, Choose, Shuffle, version, \
setmidiplayer, playfile, rescale, harmonics, keynum, hertz, divide, temper, deltas, \
scale, isnum, rescale, highran, subtract
from musx.midi.gm import Flute, MusicBox, TubularBells
print(f'version: {version}')

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /Users/taube/Music/SoundFonts/MuseScore_General.sf2")
print('OK!')

## Microtonality and MIDI

For spectral and microtonal composers the harmonic series serves as an unlimited source of inspiration.  However -- with the trivial exception of unisons and octaves -- the natural harmonic series yield intervals with no match in the 'modern' equal tempered (12-TET) scale, which, by default, MIDI performances use!

<!-- In contrast, the natural harmonic series has no equal tempered intervals (except, trivially, the octav) -- every other interval in the series deviates from equal temperment. 

WIth the (trivial) exceptions of the unison and octave, the natural harmonic series has no intervals in common with equal temperment.
By default, MIDI key numbers 0-127 will produce equal tempered sound in a MIDI performance. 

For spectral and microtonal composers the harmonic series serves as an unlimited source of inspiration. By default the MIDI key numbers 0-127 define 11 octaves of the *twelve tone equal tempered scale* (12-TET). In contrast, the natural harmonic series has no equal tempered intervals except (trivially) the octave -- every other interval in the series deviates from equal temperment. -->

<img src="support/1300px-Harmonics_to_32.png" alt="support/1300px-Harmonics_to_32.png" width='600' />

A comparison of interval ratios in the 'just' major scale and the equal tempered equivalent. Note that the numbers in the just ratios are visible as harmonic numbers in the series above:

In [ ]:
just_major =  [ round(i, 3)         for i in [1/1,  9/8, 5/4, 4/3, 3/2, 5/3, 15/8, 2/1] ]
equal_major = [ round(2**(i/12), 3) for i in [0, 2, 4, 5, 7, 9, 11, 12]]
print(f"just major scale:   {just_major}")
print(f"12-TET major scale: {equal_major}")

From the vantage point of 12-TET (MIDI), every pitch in the harmonic series involves *microtones*, i.e. tones that lie somewhere in between two adjacent equal tempered tones. This means that to perform microtones there must be an adjustment to the equal tempered pitch. In MIDI, this adjustment is made by using a *pitch bend* message.

## MIDI Pitch Bends

Since standard MIDI is 12-TET, a midi keyboard or synthesizer will not produce microtones unless modified by a pitch bend that moves the frequency of *all the notes sounding on a given channel* higher or lower by some amount.

`MidiEvent.pitch_bend(channel, bend, time=0.0)`

* Channel is the midi channel 0-15.

* Bend is a 14bit quantity (0 to 16383) defining the amount to deviate from equal temperament. The bend value is sometimes described as ranging from -8192 to +8191 with 0 being equal temperament. 

What the bend value actually maps to depends on the MIDI device you are using. The typical mapping is that the lowest bend value equals one whole-step down and the highest value represents one whole step up. But this is only a convention -- some devices will let you set the maximum bend value move an octave up, for example.

Run this example to listen to pitch bends. The pitch bends in the first example sound like individual notes because only 10 pitch bends are used over 3 seconds. The second version uses twice as many pitch bends as the first and sounds more continuous. The last two use 40 bends and sound much smoother.  The last example also demonstrates that pitch bends affect all the notes that are currently sounding on the pitch bend's channel, rather than on a per-note basis**. 

** The recent (2020) MIDI 2.0 release supports [per-note pitch bends](https://www.youtube.com/watch?v=x2QxFnsKWMQ).

In [ ]:
def bender(score, length, cycs, rhy, dur, key, amp):
    twopi = 2 * math.pi
    if not isinstance(key, list):
        key = [key]
    for i in range(length):
        if i == 0:
            for k in key:
                n = Note(time=score.now, duration=dur, pitch=k, amplitude=amp)
                score.add(n)
        a = math.sin(twopi * cycs * (i / length))
        b = int(rescale(a, -1, 1, 0, 16383))
        score.add(MidiEvent.pitch_bend(0, b, score.now))
        yield rhy

meta = MidiFile.metatrack(ins={0: Flute})
score = Score(out=Seq())
score.compose([ [0,  bender(score, 10, 1, .1, 3, 72, .8)],
                [4,  bender(score, 20, 1, .05, 3, 72, .8)],
                [8,  bender(score, 40, 1, .025, 3, 72, .8)],
                [12, bender(score, 40, 1, .025, 3, [72,76,79,81], .8)]
              ])

file = MidiFile("micromidi.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

## Musx channel tuning

As can be seen in the above example, working with pitch bends directly can be rather unwieldy! To help facilitate working with non 12-TET tunings musx provides two features that, when used together, allow composers to more easily approximate true microtonal scales and tunings. 

The first feature is musx's floating point key numbers *kkk.cc* where *kkk* is the midi key number and *cc* is interpreted as cents above the midi key *kkk*. For example, the keynum 60.5 specifies a pitch that is 50 cents (one quarter-tone) above the equal tempered tuning. Since a semitone is 100 cents, any microtonal value can be encoded as a floating point keynum.  

The second feature, called *channel tuning*, implements a fixed, microtonal setup for an entire midi file. Microtuning is activated by using the *microdivs* argument to *metatrack()*. The microdivs option quantizes the semitone into some number of micro-divisions. The default value for *microdivs* is 1, which means that the semitone is divided by 1, so no microtonal output occurs. However, if microdivs is 2 then semitone/2 = 50 cent quantization (quartertone tuning), and so on. The maxmimum number for microdivs is 16, or 6.25 cents, which is very close to the frequency limen of ~5 cents. 

Channel tuning works by claiming certain channels to use for microtonal inflection. For example, if microdivs is 2 then channel tuning claims successive *pairs* of channels for quarter-tone tuning:

channels:      | chan0 | chan1 | chan2 | chan3 | ... |
---------------|-------|-------|-------|-------|-----|
tuning (cents):| 0     | 50    | 0     | 50    | ... |
instrument:    | flute | flute | viola | viola | ... |

This means that the channels available for different instruments are 0, 2, 4, 6, 8, 10, 12, and 14, and the channels 1 3 5 6 9 11 13 15 are automatically assigned the same instrument as their left side channel but tuned one 1/4 tone higher. This means that any fractional key number value .5 or greater that musx sends to the midi file on channel 0 will be automatically routed to the second channel in the pair (channel 1). A microdivs of 16 will claim all 16 channels in order to produce microtones so the only channel available for instrument assignment will be channel 0. For more information see the micro.ipynb file in the demos directory.

The following three cells plays one octave of microtones quantized to 1/divs semitones:

In [ ]:
def play_micro_divisions(divs):

    def playmicro(score, key, rhy, divs):
        inc = 1/divs
        for i in range(12 * divs + 1):
            note = Note(time=score.now, duration=rhy, pitch=key)
            score.add(note)
            key += inc
            yield rhy
     
    track0 = MidiFile.metatrack(microdivs=divs) # divs can range 1 to 16
    track1 = Seq()
    score = Score(out=track1)
    score.compose( playmicro(score, 60, .5, divs) )
    file = MidiFile("micromidi.mid", [track0, track1]).write()
    print(f"Wrote '{file.pathname}'.")
    track0.print()
    playfile(file.pathname)
    
print(f'play_micro_divisions: {play_micro_divisions}')

This example plays floating point key numbers quantized to quarter tones (24-TET tuning):

In [ ]:
play_micro_divisions(2)

Floating point key numbers quantized to approximately 16 cents (72-TET tuning):

In [ ]:
play_micro_divisions(6)

## Working with the harmonic series

`harmonics(harmonic1, harmonic2, fund=1, reverse=False)`

The `harmonics()` function returns the harmonic series ratios between
two harmonic numbers. If 0 < harmonic1 < harmonic2 then the ratios will produce
the *overtone series*. Note that values in this series are now normalized to the first harmonic in the series, so the first number becomes 1 and each number above the first value is the ratio distance up from the first number to that harmonic.  The third argument (*fund*) is the "fundamental" for the series.  If the default value '1' is use then ratios are returned as python Fractions to keep tuning ratios more readable and as accurate as possible.

In [ ]:
harmonics(8, 16)

If (0 > harmonic1 > harmonic2) will produce [undertones](https://en.wikipedia.org/wiki/Undertone_series). See: [Harry Partch](https://en.wikipedia.org/wiki/Harry_Partch#Theory):

In [ ]:
harmonics(-8, -16)

You can convert the series to a specific fundamental in hertz using the function's 3rd argument *fund*:

In [ ]:
harmonics(8, 16, 220.0)|

In [ ]:
harmonics(-8, -16, 440.0)

Normally overtone ratios are ascending and undertones are
descending.  If reverse is True then overtones are returned in reversed order 
(overtones descending, undertones ascending):

In [ ]:
harmonics(8, 16, 220.0, reverse=True)

In [ ]:
harmonics(-8, -16, 440.0, reverse=True)

### Performing the harmonic series

To play the harmonic series using MIDI we will need to use the `keynum()` function to convert hertz values into floating point key numbers. The `playharms()` part composer transposes the harmonics between values *h1* and *h2* inclusive into list of floating point key numbers to perform:

In [ ]:
def playharms(score, fund, h1, h2, rhy, amp):
    freqs = harmonics(h1, h2, fund)
    keynums = keynum(freqs, filt=None)
    #print("freqs:", freqs, "\nkeynums:", keynums)
    for k in keynums:
        m = Note(time=score.now, duration=rhy, pitch=k, amplitude=amp)
        score.add(m)
        yield rhy
print(f"playharms: {playharms}")

Now play the overtones quantized to 1/8 tone tuning:

In [ ]:
meta = MidiFile.metatrack(microdivs=4)
score = Score(out=Seq())
score.compose([[0,    playharms(score, hertz("C4"),  8,  16, .5, .5)],
               [.5*9, playharms(score, hertz("C5"), -8, -16, .5, .5)]])
file = MidiFile("micromidi.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

This example creates a 5 voice texture spread over three octaves starting on each new prime number above the fundamental in the series, e.g. 2 3 5 7 11. The prime number also serves as the rhythmic value for each voice.

<img src="support/primeharmonics.png" width='800'/>

In [ ]:
def primevoice(score, fund, beat, harm1, harm2):
    ryth = beat * 4 
    for freq in harmonics(harm1, harm2, fund=fund*harm1):
        k = keynum(freq, filt=None)
        n = Note(time=score.now, duration=ryth, pitch=k)
        score.add(n)
        yield ryth
print(f"primevoice: {primevoice}")

Generate the score:

In [ ]:
meta = MidiFile.metatrack(microdivs=12)
score = Score(out=Seq())
score.compose([ primevoice(score, hertz("c1"), 1/p, p, p*4) for p in [2,3,5,7,11] ])
file = MidiFile("micromidi.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

## Converting harmonics into tuning ratios, floating point intervals, and scales.

Composers often want to treat the distances between harmonics as *just intervals* that can be used to transpose any pitch by a just amount. Since just intervals are ratios in the harmonic series they will always involve microtones relative to 12-TET.

The next few examples convert harmonic numbers in the fourth octave into rations and scales.
<img src="support/justdiatonic.png" width='800'/>

To convert a series of harmonic numbers into interval ratios, divide the harmonic numbers by the lowest harmonic in the series. To perform the division you can use a python comprehension or the musx `divide()` function. `divide()`  accepts floats or Fractions for the division. Fractions have the advantage of preserving the exact ratio value, e.g. 1/3 instead of 0.3333333... 

In [ ]:
fourth_octave = [8, 9, 10, 11, 12, 13, 14, 15, 16]
print(f"harmonics:    {fourth_octave}")

float_ratios = divide(fourth_octave, 8)
print(f"\nfloat ratios: {float_ratios}")

fract_ratios = divide(fourth_octave, Fraction(8,1))
print(f"\nfract ratios: {fract_ratios}")

The ratios can now easily be converted to a scale in hertz or keynum formats:

In [ ]:
print(f"scale in hertz:   {[r * 440 for r in float_ratios]}")

print(f"\nscale in keynums: {keynum([r * 440 for r in float_ratios], None ) }")

It is also possible to convert harmonic ratios into 'semitone ratios', e.g. floating point intervals *sss.cc* where *sss* is the number of semitones and *cc* is the extra cents above it. Semitone intervals can be added rather than multiplied and generally make things more simple. 

The temper function converts a ratio into a semitonal ratios or a cent value:

`temper(ratio, div=12)`

*ratio* is the ratio to convert and *div* is the divisions per octave. A *div* value of 12 will convert a ratio into a semitone interval *sss.cc*.  A value of 1200 will convert it into cents.

The semitone interval of a just fifth and fourth:

In [ ]:
fifth = temper(Fraction(3, 2))
print(f"semitones in a just fifth:  {fifth}")
fourth = temper(Fraction(4,3))
print(f"semitones in a just fourth: {fourth}")
print(f"5th+4th: {fifth+fourth}")

A *div* value of 1200 will convert the just ratio into cents, in the case of cents you should probably also round to an integer, since values less than that are not audible for humans. This example reveals that a perfect fifth in the just scale is 702 cents, two cents wider than in equal temperament (which is 700 cents).

In [ ]:
print(f"width of just fifth:   {round(temper(Fraction(3, 2), 1200))}")

print(f"width of 12-TET fifth: {round(temper(2**(7/12), 1200))}")

## Microtonal Scales

To convert semitone intervals into a scale, first calculate the interval distances between adjacent semitone values. You can use the musx function `deltas()` to get these differences:

In [ ]:
print("semitones:", semitones)
intervals = deltas(semitones)
print("\ndeltas:", intervals)
print("\ndeltas sum to:", sum(intervals))

Now that we have an octave of the scale defined as deltas we can use the *scale()* function to create the scale on any key number we want.

This example produces a 4 octave scale starting on C3:

In [ ]:
just_scale = scale(48, len(intervals)*4+1, intervals)
print(just_scale)

Play the scale with a microtonal resolution of 25 cents (100 cents/4):e

In [ ]:
def playscale(score, num, rhy, scale):
    for _ in range(num):
        m = Note(time=score.now, duration=rhy, pitch=scale.next())
        score.add(m)
        yield rhy

meta = MidiFile.metatrack(microdivs=4)
score = Score(out=Seq())
length = len(just_scale)
sec1 = playscale(score, length, .2, Cycle(just_scale))       # ascending
sec2 = playscale(score, length, .2, Shuffle(just_scale))     # random
sec3 = playscale(score, length, .2, Cycle(just_scale[::-1])) # descending
score.compose([[0 , sec1], [.2*length, sec2], [.2*length*2, sec3]])
file = MidiFile("micromidi.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

### Example: Ben Johnston's blues scale from Suite for Microtonal Piano

The piano tuning for [Ben Johnston's](https://en.wikipedia.org/wiki/Ben_Johnston_(composer)) [Suite for Microtonal Piano](https://www.google.com/search?q=Ben+Johnston%27s+Suite+for+Microtonal+Piano&oq=Ben+Johnston%27s+Suite+for+Microtonal+Piano&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIICAEQABgWGB4yCggCEAAYhgMYigUyCggDEAAYhgMYigUyCggEEAAYhgMYigUyCggFEAAYhgMYigXSAQczMTZqMGo3qAIAsAIA&sourceid=chrome&ie=UTF-8#fpstate=ive&vld=cid:720442a5,vid:Nx27dfey6iM,st:168)
is taken from a selection of overtones in the fifth octave of the harmonics series based on C. The scale includes all the notes of the 4th octave harmonic series (8-16) plus some of the new harmonics appearing in the 5th octave: 17, 19, 21 and 27. The remaining new harmonics: 23, 25, 29, 31 were omitted, yielding a 12-note just chromatic tuning scale for the piano.

```
C    C#  D   Eb  E   F   F#  G   Ab  A   Bb  B   C  
[16, 17, 18, 19, 20, 21, 22, 24, 26, 27, 28, 30, 32] (the octave)
```

The Blues movement is in D dorian and also includes notes from the D chromatic scale. Dividing the C-based scale by 9/8 yields the D dorian tuning where D is 1/1 and 2/1:

The chromatic D dorian blues scale:

```
 D    D#     E     F    F#    G    Ab    A    Bb    B    C     C#    D
[1/1, 19/18, 10/9, 7/6, 11/9, 4/3, 13/9, 3/2, 14/9, 5/3, 16/9, 17/9, 2/1]
```

The diatonic D dorian blues scale is therefore:

```
 D    E     F    G    A    Bb    C     D 
[1/1, 10/9, 7/6, 4/3, 3/2, 14/9, 16/9, 2/1]
```

Pay special attention to the blue note F (7/6) which is the true "blues third" between harmonic 6 and 7 in 4th octave of the harmonic but "transposed" to D to provide a true blues third above the just dorian tonic D. Note that a second (wider) 11/9 third above D is also featured in the scale.

Here is a comparison of the just thirds with their equal tempered counterparts:

In [ ]:
print("equal tempered major third:  ", temper(2**(4/12)))
print("just third between 9 and 11: ", temper(11/9))
print("equal tempered minor third:  ", temper(2**(3/12)))
print("blues third between 6 and 7: ", temper(7/6))

Here is a link to a video-recording plus score for Ben Johnston's  amazing "Blues" movement from his Suite for Microtonal Piano. Ben Johnston was on the composition faculty here at the UIUC from 1951 to 1986. He died on July 21, 2019, in Deerfield Wisconsin.

[Performance of "Blues", from "Suite for Microtonal Piano" by Ben Johnston](https://www.google.com/search?q=Ben+Johnston%27s+Suite+for+Microtonal+Piano&oq=Ben+Johnston%27s+Suite+for+Microtonal+Piano&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIICAEQABgWGB4yCggCEAAYhgMYigUyCggDEAAYhgMYigUyCggEEAAYhgMYigUyCggFEAAYhgMYigXSAQczMTZqMGo3qAIAsAIA&sourceid=chrome&ie=UTF-8#fpstate=ive&vld=cid:720442a5,vid:Nx27dfey6iM,st:168)

This example shows how to create Johnston's just blues scale using the procedure outlined above. The scale uses partials taken from 5th octave with many partials omitted so the scale  sounds "diatonic":

In [ ]:
#            D   E   F   F#  G   A   Bb  C   D
blues_harmonics = [18, 20, 21, 22, 24, 27, 28, 32, 36] 
print(f"blues_harmonics: {blues_harmonics}")

# Convert harmonic numbers to ratios 1/1 to 2/1 based on the lowest harmonic (18):
blues_ratios = divide(blues_harmonics, Fraction(18,1))
print(f"\nblues_ratios: {blues_ratios}")

# Convert the ratios into semitones for working with midi:
blues_semitones = temper(blues_ratios)
print(f"\nblues_semitones: {blues_semitones}")

# Convert from semitones above a starting keynumber to semitone between successive steps:
blues_intervals = deltas(blues_semitones)
print(f"\nblues_intervals: {blues_intervals}")

# Create the scale to perform
blues_scale = scale(60, 8*3+1, blues_intervals)
print(f"\nblues_scale: {blues_scale}")

Listen to the blues scale:

In [ ]:
meta = MidiFile.metatrack(microdivs=4)
score = Score(out=Seq())
num = len(blues_scale)
c1 = playscale(score, num, .25, Cycle(blues_scale))
c2 = playscale(score, num, .25, Shuffle(blues_scale))
c3 = playscale(score, num, .25, Cycle(blues_scale[::-1]))
score.compose([[0 , c1], [.25*num, c2], [.25*num*2, c3]])
file = MidiFile("micromidi.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

## Listen to the pentatonic primes

There is a beautiful pentatonic scale consisting of all the new prime numbered harmonics in the 5th octave of the harmonic series. What does this scale sound like?

Write the prime numbers in the 5th octave starting on harmonic 17 and include the octave to end it:
<!-- SOLUTION
prime_harmonics = [17, 19, 23, 29, 31, 34]
print(prime_harmonics)
-->

In [ ]:
prime_harmonics = [17, 19, 23, 29, 31, 34]
print(f"prime_harmonics: {prime_harmonics}")

Convert the harmonic numbers into ratios from 1/1 to 2/1 based on the lowest harmonic (17):

<!-- SOLUTION
prime_ratios = divide([17, 19, 23, 29, 31, 34], 17)
print(prime_ratios)
-->

In [ ]:
prime_ratios = divide([17, 19, 23, 29, 31, 34], 17)
print(f"prime_ratios: {prime_ratios}")

Convert the ratios into floating point semitones for midi output:
<!-- SOLUTION
prime_semitones = temper(prime_ratios)
print(prime_semitones)
-->

In [ ]:
prime_semitones = temper(prime_ratios)
print(f"prime_semitones: {prime_semitones}")

Convert to semitones *between* successive steps:

<!-- SOLUTION
prime_intervals = deltas(prime_semitones)
print(prime_intervals)
-->

In [ ]:
prime_intervals = deltas(prime_semitones)
print(f"prime_semitones: {prime_intervals}")

Convert the interval steps into a full three octave scale based on key number 60 and ending on keynum 96:
<!-- SOLUTION
prime_scale = scale(60, 15+1, prime_intervals)
print(prime_scale)
-->

In [ ]:
prime_scale = scale(60, 15+1, prime_intervals)
print(f"prime_scale: {prime_scale}")

The part composer to perform the scale:

In [ ]:
def prime_chime(score, numnotes, prime_scale, rhy, dur, amp, chan):
    rhy = rhy * 2
    dur = dur * 2
    rhythms = Choose([rhy, 
                      Cycle([rhy, rhy/2 ]), 
                      Cycle([rhy, rhy/3, (rhy/3)*2])], 
                     [6, 2, 1])
    size = len(prime_scale)
    for _ in range(numnotes):
        r = rhythms.next()
        n = int(rescale(highran(), 0, 1, 0, size))
        k = prime_scale[n]
        m = Note(time=score.now, duration=rhy, pitch=k, amplitude=amp, instrument=chan)
        score.add(m)
        yield rhy
        
print(f"prime_chime: {prime_chime}")

Create the two voice composition:

In [ ]:
meta = MidiFile.metatrack(ins={0: MusicBox, 4: TubularBells}, microdivs=4)
score = Score(out=Seq())
score.compose([prime_chime(score, 60, prime_scale, .25, 1, .6, 0),
               prime_chime(score, 30, subtract(prime_scale, 24) , .25*2, 1*2, .6, 4)])
file = MidiFile("micromidi.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

### Gamelan Scale Project

Try creating the Javanese pelog and slendro scales. Since there is no single tuning for these scales, try making Slendro a 5 tone scale in 5-TET tuning and Pelog a 7 tone scale in 9-TET tuning, where intervals are 1 1 2 1 1 1 2 (1). The interval steps for an octave of a 5-TET scale would be 2**(i/5) where i ranges from 0 to 5 inclusive.

<!-- 
slendro = [musx.temper(2 ** (i/5)) for i in range(6)]
pelog = [musx.temper(2 ** (i/9)) for i in [0, 1, 2, 4, 5, 6, 7, 9]]


slendro = [0.0, 2.4000000000000012, 4.8, 7.199999999999999, 9.6, 12.0]

pelog = [0.0, 1.3333333333333324, 2.666666666666666, 5.333333333333332, 
         6.666666666666667, 7.999999999999998, 9.333333333333334, 12.0]



from musx.midi.gm import MusicBox, TubularBells

#def playscale(q, n, r, scale):
#    #print(scale)
#    for k in scale:
#        #print(k)
#        m = musx.MidiNote(time=q.now, dur=r, key=k, tuning=4)
#        q.out.addevent(m)
#        yield r
        
a = musx.MidiFile.metatrack(ins={0: TubularBells}, microdivs=4)
b = musx.Seq()
q = musx.Score(out=b)
q.compose(playscale(q, 0, .4 , scale))
f = musx.MidiFile("harms1.mid", [a, b]).write()
print(f.pathname)
musx.playfile(f.pathname)
-->

In [ ]:
slendro = None
pelog = None